Paper: http://people.cs.uchicago.edu/~ravenben/publications/pdf/translearn-usenixsec18.pdf

In [1]:
!git clone https://github.com/bolunwang/translearn.git
%cd translearn

Cloning into 'translearn'...
remote: Enumerating objects: 84, done.
remote: Total 84 (delta 0), reused 0 (delta 0), pack-reused 84
Unpacking objects: 100% (84/84), done.
/content/translearn


In [2]:
!pip install -q ipdb

     |████████████████████████████████| 792 kB 20.5 MB/s 
     |████████████████████████████████| 374 kB 51.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.24 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.31.0 which is incompatible.


# Inspect dataset in HDF5

In [28]:
import h5py

In [29]:
dataset_path = "datasets/pubfig65_imagenet_test.h5"
f = h5py.File(dataset_path)

In [30]:
print("Data type:", type(f.get("X_test")[:]))
print("Shape:", f.get("X_test").shape)

Data type: <class 'numpy.ndarray'>
Shape: (65, 224, 224, 3)


In [31]:
# lb = f.get("Y_test")[:]
f.get("Y_test")[0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

## Convert numpy array to image

In [ ]:
import numpy as np
from PIL import Image
from matplotlib import cm

Convert numpy pixel array to image
- https://stackoverflow.com/a/10967471
- https://stackoverflow.com/a/2659378

In [ ]:
sample_test_images = f.get("X_test")[:]
for num, image_array in enumerate(sample_test_images):
  im = Image.fromarray(image_array)
  im.save(f"sample_imagenet_test_{num}.png")

In [ ]:
sample_test_images.shape

(65, 224, 224, 3)

## Test loading data

In [32]:
from pubfig65_vggface_mimic_penalty_dssim import *

/usr/local/lib/python3.7/dist-packages/keras/backend.py:414: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [33]:
(bottleneck_model, student_model) = load_and_build_models()

loading student model
loading teacher model
building bottleneck model


In [34]:
X, Y = load_dataset()

In [35]:
print("X shape:", X.shape)
print("Y shape:", Y.shape)

X shape: (65, 224, 224, 3)
Y shape: (65,)


In [ ]:
f.get("Y_test")[0]

In [50]:
from pubfig65_vggface_mimic_penalty_dssim import *

In [59]:
dataset = utils_translearn.load_dataset(
    DATA_FILE,
    keys=['X_test', 'Y_test'])

In [60]:
X = dataset['X_test']
Y = dataset['Y_test']

X = X.astype(np.float32)
Y = Y.astype(np.float32)

In [63]:
X = utils_translearn.preprocess(X, INTENSITY_RANGE)
Y = np.argmax(Y, axis=1)

In [64]:
import numpy as np
from PIL import Image
from matplotlib import cm

image_array = X[0]
im = Image.fromarray(np.uint8(image_array))
im.save(f"sample_imagenet_test_2.png")

In [36]:
# filter data points, keep only correctly predicted samples
print('filtering data')
X, Y = filter_data(X, Y, student_model)

filtering data
baseline accuracy of student 0.938462
X shape: (61, 224, 224, 3)
Y shape: (61,)


In [ ]:
Y_label = list(np.unique(Y))
all_pair_list = list(itertools.permutations(Y_label, 2))
pair_list = random.sample(
    all_pair_list,
    min(NB_PAIR, len(all_pair_list)))

In [ ]:
pair_list

[(2, 21), (6, 40)]

In [ ]:
(source, target) = pair_list[0]
# sample images
(X_source, Y_source, X_target, Y_target) = select_source_target(
  X, Y, source, target)

In [ ]:
X_target.shape

(1, 224, 224, 3)

In [ ]:
def load_and_build_models(student_model_file=STUDENT_MODEL_FILE,
                          teacher_model_file=TEACHER_MODEL_FILE,
                          cutoff_layer=CUTOFF_LAYER):

    # load the student model
    print('loading student model')
    student_model = load_model(student_model_file)

    print('loading teacher model')
    teacher_model = load_model(teacher_model_file)

    # load the bottleneck model
    print('building bottleneck model')
    import ipdb; ipdb.set_trace()
    bottleneck_model = Model(teacher_model.input,
                             teacher_model.layers[cutoff_layer - 1].output)
    bottleneck_model.compile(loss='categorical_crossentropy',
                             optimizer='adam',
                             metrics=['accuracy'])

    return bottleneck_model, student_model

In [ ]:
(bottleneck_model, student_model) = load_and_build_models()

loading student model
loading teacher model



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 332, in set_trace
    sys.settrace(self.trace_dispatch)



building bottleneck model
> <ipython-input-67-fecc055b9e2c>(15)load_and_build_models()
     14     import ipdb; ipdb.set_trace()
---> 15     bottleneck_model = Model(teacher_model.input,
     16                              teacher_model.layers[cutoff_layer - 1].output)

ipdb> teacher_model
ipdb> teacher_model.layers
[<keras.engine.input_layer.InputLayer object at 0x7f3a6a30e910>, <keras.layers.convolutional.Conv2D object at 0x7f3ad9183750>, <keras.layers.core.activation.Activation object at 0x7f3a64c86790>, <keras.layers.convolutional.Conv2D object at 0x7f3ad91b3f10>, <keras.layers.core.activation.Activation object at 0x7f3a64c86410>, <keras.layers.pooling.MaxPooling2D object at 0x7f3a64df1790>, <keras.layers.convolutional.Conv2D object at 0x7f3a6a30e8d0>, <keras.layers.core.activation.Activation object at 0x7f3a64d9d890>, <keras.layers.convolutional.Conv2D object at 0x7f3a64d9df10>, <keras.layers.core.activation.Activation object at 0x7f3a64d9d8d0>, <keras.layers.pooling.MaxPooling2D


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 357, in set_quit
    sys.settrace(None)



BdbQuit: ignored

# Prepare to run

Download exported model:
- vggface.h5
- pubfig65_vggface_trans_nbtrain_90.h5

In [3]:
!pip install -q gdown

In [4]:
!mkdir ./models
!gdown "https://drive.google.com/uc?id=1f2wV9nOLVKFM6JmeiCm3n05BmWZIQHyl" -O "models/pubfig65_vggface_trans_nbtrain_90.h5"
!gdown "https://drive.google.com/uc?id=1hQ-cVn1hy85rXSmgKFa-H0aIMUsVr1Tz" -O "models/vggface.h5"
# !cp datasets/pubfig65_imagenet_test.h5 models/

Downloading...
From: https://drive.google.com/uc?id=1f2wV9nOLVKFM6JmeiCm3n05BmWZIQHyl
To: /content/translearn/models/pubfig65_vggface_trans_nbtrain_90.h5
100% 538M/538M [00:05<00:00, 90.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1hQ-cVn1hy85rXSmgKFa-H0aIMUsVr1Tz
To: /content/translearn/models/vggface.h5
100% 580M/580M [00:04<00:00, 122MB/s]


In [ ]:
# Check CUDA version
# https://stackoverflow.com/questions/9727688/how-to-get-the-cuda-version
import tensorflow as tf
from tensorflow.python.platform import build_info as build
print(f"tensorflow version: {tf.__version__}")
print(f"Cuda Version: {build.build_info['cuda_version']}")
print(f"Cudnn version: {build.build_info['cudnn_version']}")

tensorflow version: 2.7.0
Cuda Version: 11.1
Cudnn version: 8


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb

!ls  # Check if required cuda 9.0 amd64-deb file is downloaded

!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb

!ls /var/cuda-repo-9-0-local | grep .pub

!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub

!apt-get update

!sudo apt-get install cuda-9.0

--2022-01-03 04:08:12--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb [following]
--2022-01-03 04:08:12--  https://developer.nvidia.com/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?TMV1g2WVlpv_Jrxec4h44te6sKcgb8MIDzKXM_7speDL07Bg5CfH-jsyLVQW9_P3gSeNaOl1KWX2-1SFcKdXNvQOKk_zN

In [ ]:
!apt install python2.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python2.7 is already the newest version (2.7.17-1~18.04ubuntu1.6).
python2.7 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 100 not upgraded.


In [ ]:
!python2.7 -m pip install keras==2.2.0 numpy==1.14.0 tensorflow-gpu==1.8.0 h5py==2.8.0 ipdb

# Attack

In [ ]:
%%time
!python2.7 pubfig65_vggface_mimic_penalty_dssim.py

Using TensorFlow backend.
loading student model
2022-01-02 15:09:30.939803: W tensorflow/core/framework/allocator.cc:101] Allocation of 411041792 exceeds 10% of system memory.
/usr/local/lib/python2.7/dist-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
loading teacher model
2022-01-02 15:09:34.487043: W tensorflow/core/framework/allocator.cc:101] Allocation of 411041792 exceeds 10% of system memory.
building bottleneck model
loading attacker
Attacker loaded
loading dataset
filtering data
baseline accuracy of student 0.938462
X shape: (61, 224, 224, 3)
Y shape: (61,)
INFO: processing pair #0 source: 62, target: 33
1 batches in total
processing batch 0 at 2022-01-02 15:09:43.296832
START:     Total loss: 2.1263E+02; perturb: 0.000000 (0.00% over, raw: 0.000000); sim: 212.627869
ITER    0: Total loss: 2.1127E+02; perturb: 0.00

# Fingerprinting

In [ ]:
%%time
!python2.7 pubfig65_fingerprint_vggface.py

Using TensorFlow backend.
loading student model
2022-01-01 14:44:25.652038: W tensorflow/core/framework/allocator.cc:101] Allocation of 411041792 exceeds 10% of system memory.
2022-01-01 14:44:25.652596: W tensorflow/core/framework/allocator.cc:101] Allocation of 67108864 exceeds 10% of system memory.
/usr/local/lib/python2.7/dist-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
loading teacher model
2022-01-01 14:44:29.126593: W tensorflow/core/framework/allocator.cc:101] Allocation of 411041792 exceeds 10% of system memory.
2022-01-01 14:44:29.126653: W tensorflow/core/framework/allocator.cc:101] Allocation of 67108864 exceeds 10% of system memory.
2022-01-01 14:44:29.126698: W tensorflow/core/framework/allocator.cc:101] Allocation of 42958848 exceeds 10% of system memory.
building bottleneck model
loading attacker
Attacker

In [ ]:
%%time
!python2.7 pubfig65_fingerprint_vgg16.py

Using TensorFlow backend.
loading student model
2022-01-01 15:05:00.018879: W tensorflow/core/framework/allocator.cc:101] Allocation of 411041792 exceeds 10% of system memory.
/usr/local/lib/python2.7/dist-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
loading teacher model
553476096/553467096 [==============================] - 57s 0us/step
2022-01-01 15:05:59.252058: W tensorflow/core/framework/allocator.cc:101] Allocation of 411041792 exceeds 10% of system memory.
building bottleneck model
loading attacker
Attacker loaded
building fingerprinting input
1 batches in total
processing batch 0 at 2022-01-01 15:06:01.498691
START:     Total loss: 6.4014E+01; perturb: 0.000000 (0.00% over, raw: -0.000000); sim: 64.014488
ITER    0: Total loss: 6.3486E+01; perturb: 0.000000 (0.00% over, raw: -0.000000); sim: 63.486332
ITER  200: 

# Patch neural networks

In [ ]:
%%time
!python2.7 pubfig65_patch_neuron_distance.py

# Extension

In [ ]:
!gdown "https://drive.google.com/uc?id=1JibtwaJ0PFynllvMVzqJBOJpMZt5J5kL" -O "models/iris_vgg16_trans.h5"
!gdown "https://drive.google.com/uc?id=1-b4Swcr0XDZ2Car9Ed9E9-9Vgciqa87r" -O "models/gtsrb_vgg16_trans.h5"
!gdown "https://drive.google.com/uc?id=1BR3g0Sq5WoqcaJe5GRvawfxDKmLoSLW7" -O "models/flower_resnet50_trans.h5"

# Experiment on Food classification

In [5]:
!mkdir ./food_models/
!gdown "https://drive.google.com/uc?id=1QlwYG5HEU3SIG1FMVp_7VPZ1gyiGNwT4" -O "food_models/extractor__tl_model_v1.weights.best.hdf5"
!gdown "https://drive.google.com/uc?id=1-9VarST9tJkHFVbstK7UlbxzntfMPJP2" -O "food_models/fine_tuned__tl_model_v1.weights.best.hdf5"
!gdown "https://drive.google.com/uc?id=1qvP9MEyDLacUSBGOogTpN8UCS8OZav04" -O "food_models/vgg16.h5"

Downloading...
From: https://drive.google.com/uc?id=1QlwYG5HEU3SIG1FMVp_7VPZ1gyiGNwT4
To: /content/translearn/food_models/extractor__tl_model_v1.weights.best.hdf5
100% 1.34G/1.34G [00:29<00:00, 46.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-9VarST9tJkHFVbstK7UlbxzntfMPJP2
To: /content/translearn/food_models/fine_tuned__tl_model_v1.weights.best.hdf5
100% 1.36G/1.36G [00:28<00:00, 47.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qvP9MEyDLacUSBGOogTpN8UCS8OZav04
To: /content/translearn/food_models/vgg16.h5
100% 554M/554M [00:09<00:00, 56.4MB/s]


Download and load food dataset

In [6]:
!wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
!tar xzf food-101.tar.gz

--2022-01-05 14:43:01--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2022-01-05 14:43:01--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: ‘food-101.tar.gz’

food-101.tar.gz     100%[===================>]   4.65G  30.7MB/s    in 2m 42s  

2022-01-05 14:45:44 (29.4 MB/s) - ‘food-101.tar.gz’ saved [4996278331/4996278331]



## Testing

\################################## Start test \##################################

### Inspect model

In [ ]:
food_student_model = load_model("food_models/extractor__tl_model_v1.weights.best.hdf5")

In [ ]:
from pprint import pprint
pprint(food_student_model.layers)
pprint(len(food_student_model.layers))

24


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
model=VGG16(include_top=True , weights="imagenet")
model.save("vgg16.h5")

In [ ]:
!cp vgg16.h5 

cp: missing destination file operand after 'vgg16.h5'
Try 'cp --help' for more information.


In [ ]:
pprint(model.layers)
pprint(len(model.layers))

23


In [ ]:
input_shape = (224, 224, 3)
conv_base = VGG16(include_top=False,
                  weights='imagenet', 
                  input_shape=input_shape)
pprint(conv_base.layers)
pprint(len(conv_base.layers))

58900480/58889256 [==============================] - 1s 0us/step
19


### Prepare dataset

In [7]:
# Move data from images to images/train or images/test: 
import shutil
from collections import defaultdict
import json
from pathlib import Path
import os

def split_dataset(root_food_path):
    """Takes in the path for food-101 directory and creates train/test dirs of images"""
    data_paths = {
        'train': root_food_path/'meta/train.json',
        'test': root_food_path/'meta/test.json'
    }
    for data_type, meta_path in data_paths.items():
        # Make the train/test dirs
        os.makedirs(root_food_path/data_type, exist_ok=True)
        
        # Read the meta files. 
        # These are loaded as a dict of food names with a list of image paths
        # E.g. {"<food_name>": ["<food_name>/<image_num>", ...], ...}
        food_images = json.load(open(meta_path, 'r'))
        
        for food_name, image_paths in food_images.items():
            # Make food dir in train/test dir
            os.makedirs(root_food_path/data_type/food_name, exist_ok=True)
            
            # Move images from food-101/images to food-101/train (or test)
            for image_path in image_paths:
                image_path = image_path + '.jpg'
                shutil.move(root_food_path/'images'/image_path, root_food_path/data_type/image_path)

In [8]:
import os
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np

In [9]:
BATCH_SIZE = 64

train_generator = ImageDataGenerator(rotation_range=90, 
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input) # VGG16 preprocessing

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessing

In [10]:
# download_dir = Path('/content/')
download_dir = Path('.')

split_dataset(download_dir/'food-101')

In [11]:
train_data_dir = download_dir/'food-101/train'
test_data_dir = download_dir/'food-101/test'

class_subset = sorted(os.listdir(download_dir/'food-101/images'))[:10] # Using only the first 10 classes

In [12]:
traingen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=42)

validgen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_generator.flow_from_directory(test_data_dir,
                                             target_size=(224, 224),
                                             class_mode=None,
                                             classes=class_subset,
                                             batch_size=1,
                                             shuffle=True,
                                             seed=42)

Found 6380 images belonging to 10 classes.
Found 1120 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [25]:
tmp = next(testgen)

In [26]:
np.argmax(tmp)

2004

In [43]:
np.mean(f.get("X_test")[0])

163.0399859162415

In [44]:
np.mean(tmp[0])

25.416185

In [15]:
labels = testgen.labels

In [49]:
import numpy as np
from PIL import Image
from matplotlib import cm

image_array = f.get("X_test")[0]
im = Image.fromarray(np.uint8(image_array))
im.save(f"sample_imagenet_test_0.png")

In [18]:
labels[:300]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], d

In [ ]:
def load_food():
  """Specific function for loading food dataset"""


### Check model classification

In [ ]:
food_student_model = load_model("food_models/extractor__tl_model_v1.weights.best.hdf5")

\################################## End test \##################################

## Conducting

In [ ]:
# Move data from images to images/train or images/test: 
import shutil
from collections import defaultdict
import json
from pathlib import Path
import os

def split_dataset(root_food_path):
    """Takes in the path for food-101 directory and creates train/test dirs of images"""
    data_paths = {
        'train': root_food_path/'meta/train.json',
        'test': root_food_path/'meta/test.json'
    }
    for data_type, meta_path in data_paths.items():
        # Make the train/test dirs
        os.makedirs(root_food_path/data_type, exist_ok=True)
        
        # Read the meta files. 
        # These are loaded as a dict of food names with a list of image paths
        # E.g. {"<food_name>": ["<food_name>/<image_num>", ...], ...}
        food_images = json.load(open(meta_path, 'r'))
        
        for food_name, image_paths in food_images.items():
            # Make food dir in train/test dir
            os.makedirs(root_food_path/data_type/food_name, exist_ok=True)
            
            # Move images from food-101/images to food-101/train (or test)
            for image_path in image_paths:
                image_path = image_path + '.jpg'
                shutil.move(root_food_path/'images'/image_path, root_food_path/data_type/image_path)

In [ ]:
from pathlib import Path

download_dir = Path('/content/translearn/')

split_dataset(download_dir/'food-101')

In [ ]:
import os

class_subset = sorted(os.listdir(download_dir/'food-101/images'))[:10]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

BATCH_SIZE = 32

train_generator = ImageDataGenerator(
    rescale=1/255.,              # normalize pixel values between 0-1
    brightness_range=[0.1, 0.7], # specify the range in which to decrease/increase brightness
    width_shift_range=0.5,       # shift the width of the image 50%
    rotation_range=90,           # random rotation by 90 degrees
    horizontal_flip=True,        # 180 degree flip horizontally
    vertical_flip=True,          # 180 degree flip vertically
    validation_split=0.15        # 15% of the data will be used for validation at end of each epoch
)

# test for integration with adversarial model
testgen = train_generator.flow_from_directory(
    download_dir/'food-101/test',
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=class_subset,
    subset='validation',
    shuffle=True,
    seed=42
)

Found 370 images belonging to 10 classes.


In [ ]:
data_list = []
batch_index = 0

while batch_index <= testgen.batch_index:
    data = testgen.next()
    data_list.append(data[0])
    batch_index = batch_index + 1

# now, data_array is the numeric data of whole images
data_array = np.asarray(data_list)

Prepare model for attacking

In [ ]:
from pathlib import Path

food_classifier_path = Path("/content/translearn/food_models/")
# Deep-layer feature extractor
dlfe_model = food_clssifier_path/"extractor__tl_model_v1.weights.best.hdf5"

# Mid-layer feature extractor (K=N-3)
mlfe_model = food_clssifier_path/"fine_tuned__tl_model_v1.weights.best.hdf5"